# Determining Edge Changes and Splits

### 0 - Imports, Global Paramters & Helper Functions

In [1]:
import random

import networkx as nx
import numpy as np
from pyvis.network import Network

NUMBER_ASes = 200
NUMBER_ALLIES = 3
ATTACK_VOLUME = 100
ALLIES_SCRUBBING_CAPABILITIES = [20, 5, 18]

In [2]:
def gen_pyvis_network(G):
    """
    For Plotting. TODO
    """
    net = Network(
        notebook=True, 
        directed=True, 
        height='1000px', 
        width="100%"
    )
    net.inherit_edge_colors(False)
    net.from_nx(G)
    return net

def assign_attributes(
    Graph:nx.classes.graph.Graph, 
    victim_node:int, 
    adversary_node:int,
    helper_nodes:list = [],
    type_to_value:dict = {"T":30, "M": 20, "C":10, "CP":10}
):
    """
    Given a graph, the victim and adversary node (and optionally a set of helpers), this function will assign
    an array of attributes to these nodes.
    
    Args:
        Graph: the networkx graph object
        victim_node: the number of the node representing the destination of the DDoS attack traffic
        adversary_node: the number of the node representing the source of the DDoS attack traffic
        helper_nodes: a list of integers, representing the set of all helpers
    
    """
    for node_indx in range(len(Graph.nodes)):
        # colors according to attack/defense role
        if node_indx == victim_node:
            Graph.nodes[node_indx]["color"] = "green"
            Graph.nodes[node_indx]["victim"] = True
        elif node_indx == adversary_node:
            Graph.nodes[node_indx]["color"] = "red"
            Graph.nodes[node_indx]["adversary"] = True
        elif node_indx in helper_nodes:
            Graph.nodes[node_indx]["color"] = "blue"
            Graph.nodes[node_indx]["helper"] = True
        else:
            Graph.nodes[node_indx]["color"] = "darkgrey"
            
        # size according to type
        Graph.nodes[node_indx]["value"] = type_to_value[Graph.nodes[node_indx]["type"]]
            
    return Graph

### 1 - Graph Generation

In [3]:
def to_directed_via_BFS(
    G:nx.classes.graph.Graph, 
    victim:int
):
    """
    Used to make an undirected Graph with a victim node into a directed graph, such that
    the resulting graph is a sensible assignment for a directed, acyclic graph with a sink,
    representing the victim node. It represents the traffic flow with a destination located in
    the victim AS.
    """
    
    # representing the queue through a list and the pop(0) and append() methods
    Q = [victim]
    # for indicating, whether nodes were already explored a not
    explored = [False]*len(G.nodes)
    # for remembering which edges to remove and add
    edges_to_remove = []
    edges_to_add = []
    
    while Q:
        # assign the current node by dequeuing an element
        current = Q.pop(0)
        
        # consider all neighbors of the current node
        for neighbor in G.neighbors(current):
            
            if not explored[neighbor]:
                Q.append(neighbor)
                edges_to_remove.append((current, neighbor))
                edges_to_add.append((neighbor, current))
        #mark this node as explored
        explored[current] = True        
    
    # make G into a directed graph
    G = G.to_directed()

    # remove and add the edges, after checking if this action is legal
    for u,v in edges_to_remove:
        if (u, v) in G.edges:
            G.remove_edge(u, v)
    for u,v, in edges_to_add:
        if not (u, v) in G.edges:
            G.add_edge(u, v)
        
    return G


def generate_directed_AS_graph(
    nr_ASes:int, 
    nr_allies:int
):
    # generate the an undirected graph, whose topology is close to the AS network
    G = nx.random_internet_as_graph(NUMBER_ASes)

    # get the list of customers and content-providers
    customers_and_cps = [indx for indx in range(NUMBER_ASes) if G.nodes[indx]["type"] in ["C", "CP"]]

    # from this list, randomly select the victim, adversary and allies
    selected = random.sample(customers_and_cps, NUMBER_ALLIES+2)
    victim = selected[0]
    adversary = selected[1]
    allies = selected[2:]

    # assign attributes (e.g. color)
    G = assign_attributes(G, victim, adversary, allies)
    
    # change it to a directed, acyclic graph, with the victim as a sink
    G = to_directed_via_BFS(G, victim)
    
    return G, victim, adversary, allies


G, victim, adversary, allies = generate_directed_AS_graph(NUMBER_ASes, NUMBER_ALLIES)
# vizualize
#net = gen_pyvis_network(G)
#net.show("base_graph_undirected.html")


### 1.5 - Pruning the Graph
Pruning means that from each node we will remove some outwards edges. To be specific, we will determine the length of the path towards the victim for each edge, and only the take the shortest ones.

In [4]:
def BGP_prune_graph(
    G:nx.classes.graph.Graph,
    victim:int
):
    # make a copy to not mingle with the original graph
    G_pruned = G.copy()
    
    # get a list of all nodes, minus the victim node (it has only incoming connections)
    all_nodes = list(G_pruned.nodes)
    all_nodes.remove(victim)
    
    nr_edges_pruned = 0
    
    # go through each node
    for node in all_nodes:
        # get a list of all outward pointing edges
        outward_edges = list(G_pruned.out_edges(node))

        # then for each, determine the length of the shortest path
        costs = []
        for _, next_node in outward_edges:
            costs.append(len(list(nx.shortest_simple_paths(G_pruned, next_node, victim))[0]))

        # then remove all the ones who dont belong to the set of shortest
        shortest_path_length = min(costs)
        delete = [indx for indx, cost in enumerate(costs) if cost != shortest_path_length]
        for delete_indx in delete:
            u, v = outward_edges[delete_indx]
            G_pruned.remove_edge(u, v)
            nr_edges_pruned += 1
            
    print(f"Pruned {nr_edges_pruned} edges!")
            
    return G_pruned

G_pruned = BGP_prune_graph(G, victim)

#TODO DO WE WANT THIS
G = G_pruned
#net = gen_pyvis_network(G)
#net.show("pruned_init_graph.html")

Pruned 52 edges!


### 2 - Caculating Spit/Merge Costs

In [5]:
def calculate_merge_split_cost(
    G:nx.classes.graph.Graph,
    victim:int, 
    adversary:int,
    attack_path:list,
    ally:int,
    ally_path:list,
    step_cost:int = 2,
    split_step_cost:int = 1,
    change_cost:int = 5,
    unwanted_change_cost:int = 50,
    return_shortest_path_and_prime_graph:bool = False,
):
    # start by initializing G prime as a copy of G
    G_prime = G.copy()
    
        
    # set the cost of travelling along the attack path to step cost
    for u, v in zip(attack_path[:-1], attack_path[1:]): # TODO IS THIS IRRELEVANT NOW
        G_prime[u][v]["weight"] = step_cost
        G_prime[u][v]["edge_origin"] = "original_attack_path"

    # then set the cost of all other edges to split_step_cost
    for u,v in G_prime.edges:
        if not "edge_origin" in G_prime[u][v]:
            G_prime[u][v]["weight"] = split_step_cost
            G_prime[u][v]["edge_origin"] = "split_original"
    
    # for each edge, add the opposite edge with cost change_cost
    edges = list(G_prime.edges) # since it is changing during the loop
    for u,v in edges:
        G_prime.add_edge(v, u)
        G_prime[v][u]["weight"] = change_cost + split_step_cost
        G_prime[v][u]["edge_origin"] = "changed"
    """
    # set the reverse of the ally path to split_step_cost
    for u, v in zip(ally_path[:-1], ally_path[1:]):
        G_prime[v][u]["weight"] = split_step_cost
        G_prime[v][u]["edge_origin"] = "ally_changed"
    """
    
    # set the unwanted changes, i.e., going through the victim
    for (u, v) in G_prime.in_edges(victim):
        G_prime[u][v]["weight"] = unwanted_change_cost
        G_prime[u][v]["edge_origin"] = "no_change_wanted"
    
    # caculate the shortest path from adversary to ally
    shortest_paths_generator = nx.shortest_simple_paths(G_prime, adversary, ally, weight="weight")
    shortest_path = next(shortest_paths_generator)

    # go through it, and calculate the associated cost and check whether a edge was changed
    # for the first time a changed edge was used, note the vertex it came from, this is the splitter
    total_cost = 0
    edge_changes = []
    used_edges = [] # also contains the edges that are used, but not changes, in 
                    # contrast to edge_changes, that only contains the changes
                    # edges on the path
    for u, v in zip(shortest_path[:-1], shortest_path[1:]):
        associated_edge_data = G_prime.get_edge_data(u, v)
        total_cost += associated_edge_data["weight"]
        used_edges.append((u, v))
        if "change" in associated_edge_data["edge_origin"]:
            edge_changes.append((u, v))
    
    # determine the node responsible for the split
    splitter = edge_changes[0][0]

    return total_cost, splitter, edge_changes, used_edges
    
    
    
def build_cost_and_split_matrices(
    G:nx.classes.graph.Graph,
    victim:int, 
    adversary:int, 
    allies:list
):
    """
    Given that the "calculate_merge_split_cost" function is implemeted and implemented correctly, this 
    function will calculate the costs of trying to divert attack traffic to an ally. Explicitly, it will return
    a matrix with the cost of doing so for each (attack_path, ally) pair, together with the associated node 
    that is responsible for splitting the traffic, if there is such a node.
    
    Args:
    
    
    Returns:
        cost_matrix:             contains the cost, rows represent helpers, columns the attack paths
        split_node_matrix:       contains the node responsible for splitting, rows represent helpers, 
                                     columns the attack paths
    """

    # determine all attack paths
    attack_paths = list(nx.all_simple_paths(G, adversary, victim))
    
    # determine all support paths
    allies_paths = [list(nx.all_simple_paths(G, ally, victim)) for ally in allies]

    # matrix for noting the merge cost and the splitting node if there is any
    # also a list for saving the done changes
    cost_matrix = np.zeros((len(allies), len(attack_paths))).astype("int")
    split_node_matrix = np.zeros(cost_matrix.shape).astype("int")
    changes_list = [[None for _ in range(len(attack_paths))] for _ in range(len(allies))]
    used_edges_list = [[None for _ in range(len(attack_paths))] for _ in range(len(allies))]
    # pair each attack flow with each helper
    for attack_path_indx, attack_path in enumerate(attack_paths):
        for ally, ally_paths in enumerate(allies_paths):
            
            # since one ally may have multiple paths, and we are only interested in the one with the
            # lowest cost, we will collected them
            cost_collector = []
            split_collector = []
            change_collector = []
            used_edges_collector = []
            
            # go through all possible paths of the ally to the victim
            for ally_path in ally_paths:

                # and calculate the cost for merge/splitting and the splitting node if there is any
                cost, splitter, changes, used_edges = calculate_merge_split_cost(G, victim, adversary, attack_path, allies[ally], ally_path)
                # append the to the corresponding collectors
                cost_collector.append(cost)
                split_collector.append(splitter)
                change_collector.append(changes)
                used_edges_collector.append(used_edges)
            # determine the best split, according to lowest cost, and write it together with the associated
            # splitting node into the previously calculated matrices
            cost_matrix[ally, attack_path_indx] = min(cost_collector)
            split_node_matrix[ally, attack_path_indx] = split_collector[cost_collector.index(min(cost_collector))]
            changes_list[ally][attack_path_indx] = change_collector[cost_collector.index(min(cost_collector))]
            used_edges_list[ally][attack_path_indx] = used_edges_collector[cost_collector.index(min(cost_collector))]
            
    return cost_matrix, split_node_matrix, changes_list, used_edges_list
    
cost_matrix, split_node_matrix, changes_list, used_edges_list = build_cost_and_split_matrices(G, victim, adversary, allies) 

cost_matrix, split_node_matrix, changes_list, used_edges_list

(array([[21, 21],
        [15, 15],
        [16, 15]]),
 array([[0, 7],
        [0, 0],
        [7, 7]]),
 [[[(0, 1), (8, 33), (33, 62)], [(7, 4), (4, 33), (33, 62)]],
  [[(0, 1), (2, 138)], [(0, 1), (2, 138)]],
  [[(7, 24), (24, 64)], [(7, 24), (24, 64)]]],
 [[[(137, 0), (0, 1), (1, 8), (8, 33), (33, 62)],
   [(137, 0), (0, 7), (7, 4), (4, 33), (33, 62)]],
  [[(137, 0), (0, 1), (1, 2), (2, 138)], [(137, 0), (0, 1), (1, 2), (2, 138)]],
  [[(137, 0), (0, 7), (7, 24), (24, 64)],
   [(137, 0), (0, 7), (7, 24), (24, 64)]]])

### 3 - Assignment Problem

In [6]:
def ally_assignment_problem(
    cost_matrix:np.ndarray, 
    split_node_matrix:np.ndarray,
    changes_list:list,
    nr_allies:int,
):
    """
    TODO: We dont consider
        1. if one change in a solution influences other solutions
    """
    
    # for collecting all the final changes to be done
    changes_to_be_done = []
    used_allies = []
    associated_costs = []
    used_splits = []
    
    # for each attack flow, check the ally that has the lowest cost for helping
    for attack_flow_indx in range(cost_matrix.shape[1]):
        
        # get all costs of the helpers
        ally_costs = cost_matrix[:, attack_flow_indx]
        
        # NOTE: if there a multiple solution with the same cost, choose randomly one ally
        ally_min_value = np.min(ally_costs)
        lowest_allies = [ally_indx for ally_indx in range(len(ally_costs)) if ally_costs[ally_indx] == ally_min_value]
        chosen_ally = random.choice(lowest_allies)
        
        # note the parameteres
        used_allies.append(chosen_ally)
        associated_costs.append(ally_min_value)
        changes_to_be_done.append(changes_list[chosen_ally][attack_flow_indx])
        used_splits.append(split_node_matrix[chosen_ally, attack_flow_indx])

    # determine all the unique allies used
    used_allies_unique = list(set(used_allies))
    print("Changes, to be done", changes_to_be_done)
    if len(used_allies_unique) != nr_allies:
        # if not all allies were used, determine an attack flow were the addition of the not
        # ally causes the least increase in cost
        unused_allies = [ally_indx for ally_indx in range(nr_allies) if not ally_indx in used_allies_unique]

        for unused_ally in unused_allies:
            print("Unused Ally", allies[unused_ally])
            cost_difference = [cost_matrix[unused_ally][attack_flow_indx] - associated_costs[attack_flow_indx] for attack_flow_indx in range(cost_matrix.shape[1])]
            min_cost_difference = np.min(cost_difference)
            lowest_cost_differences_indices = [attack_flow_indx for attack_flow_indx in range(cost_matrix.shape[1]) if cost_difference[attack_flow_indx] == min_cost_difference]
            chosen_attack_flow = random.choice(lowest_cost_differences_indices)

            # mark it down, add the new change, dont delete the old one, because if there is only one attack flow, all should pariticpate
            used_allies.append(unused_ally)
            associated_costs.append(cost_matrix[unused_ally][chosen_attack_flow])
            changes_to_be_done.append(changes_list[unused_ally][chosen_attack_flow])
            used_splits.append(split_node_matrix[chosen_ally, attack_flow_indx])
            print("Changes, to be done", changes_to_be_done)
    # make a list of all the changes to be done
    final_changes = [item for sublist in changes_to_be_done for item in sublist]
    final_changes = list(set(final_changes))
    
    # make split uniques
    used_splits_unique = list(set(used_splits))
    
    return final_changes, used_splits_unique    
        
final_changes, used_splits = ally_assignment_problem(cost_matrix, split_node_matrix, changes_list, NUMBER_ALLIES)

print(final_changes)
print(used_splits)

Changes, to be done [[(0, 1), (2, 138)], [(7, 24), (24, 64)]]
Unused Ally 62
Changes, to be done [[(0, 1), (2, 138)], [(7, 24), (24, 64)], [(0, 1), (8, 33), (33, 62)]]
[(0, 1), (33, 62), (8, 33), (24, 64), (2, 138), (7, 24)]
[0, 7]


### 4 - Apply Changes and Determine Associated Cost

In [7]:
"""
def apply_changes(
    G:nx.classes.graph.Graph,
    list_of_edge_changes:list,
    used_edges_list:list,
    used_splits:list,
    indication_color:str = "orange"
):
    G_final = G.copy()

    # turn the indicated edges, and also change their color
    for u, v in list_of_edge_changes:
        G_final.remove_edge(v, u)
        G_final.add_edge(u, v)
        G_final[u][v]["changed_edge"] = True

    # mark all the used edges (contains changed and unchanged) for the split path
    for split_path in used_edges_list:
        for u, v in split_path[0]:
            G_final[u][v]["color"] = indication_color
            G_final[u][v]["split_path"] = True
        
    # indicate the splitter using the used_splits list
    for node in used_splits:
        G_final.nodes[node]["color"] = indication_color
        G_final.nodes[node]["is_splitting"] = True
        
    return G_final

def indicate_attack_path(
    G_init:nx.classes.graph.Graph,
    adversary:int,
    victim:int,
    indication_color:str = "deeppink"
):
    G = G_init.copy()
    # use inbuild function to see all the paths that lead from adversary to victim
    all_attack_paths = list(nx.all_simple_paths(G, adversary, victim))

    # go through each of them, and color any non-colored nodes and edges
    for attack_path in all_attack_paths:
        for u, v in zip(attack_path[:-1], attack_path[1:]):
            if G.nodes[u]["color"] == "darkgrey":
                #G.nodes[u]["color"] = indication_color
                G.nodes[u]["opacity"] = 0.75 #doesnt work
            if not "color" in G[u][v]:
                G[u][v]["color"] = indication_color
                
    return G

G_final = apply_changes(G, final_changes, used_edges_list, used_splits)
G_final_w_color = indicate_attack_path(G_final, adversary, victim)
net_final = gen_pyvis_network(G_final_w_color)
net_final.show("final_changes_G.html")
"""

'\ndef apply_changes(\n    G:nx.classes.graph.Graph,\n    list_of_edge_changes:list,\n    used_edges_list:list,\n    used_splits:list,\n    indication_color:str = "orange"\n):\n    G_final = G.copy()\n\n    # turn the indicated edges, and also change their color\n    for u, v in list_of_edge_changes:\n        G_final.remove_edge(v, u)\n        G_final.add_edge(u, v)\n        G_final[u][v]["changed_edge"] = True\n\n    # mark all the used edges (contains changed and unchanged) for the split path\n    for split_path in used_edges_list:\n        for u, v in split_path[0]:\n            G_final[u][v]["color"] = indication_color\n            G_final[u][v]["split_path"] = True\n        \n    # indicate the splitter using the used_splits list\n    for node in used_splits:\n        G_final.nodes[node]["color"] = indication_color\n        G_final.nodes[node]["is_splitting"] = True\n        \n    return G_final\n\ndef indicate_attack_path(\n    G_init:nx.classes.graph.Graph,\n    adversary:int,\n

### 4 - Apply Change, Decide Splits, Color

In [8]:
def apply_changes(
    G:nx.classes.graph.Graph,
    list_of_edge_changes:list,
    used_edges_list:list,
    used_splits:list,
    indication_color:str = "orange"
):
    G_final = G.copy()

    # turn the indicated edges, and also change their color
    for u, v in list_of_edge_changes:
        G_final.remove_edge(v, u)
        G_final.add_edge(u, v)
    return G_final

def set_splits(G_init, adversary, used_edges_list, attack_volume, ally_capabilites):
    G = G_init.copy()
    
    # set the default attack volume for all nodes and edges
    for node in G.nodes:
        G.nodes[node]["attack_vol"] = 0
    for u, v in G.edges:
        G[u][v]["split_perc"] = 0
    
    # set the attack volume of the starting node
    G.nodes[adversary]["attack_vol"] = attack_volume
    
    # calculate the amount the victim has to cover
    victim_traffic = attack_volume - sum(ally_capabilites)
    # calculate the shortest path to the victim from the adversary
    victim_path = list(nx.shortest_simple_paths(G, adversary, victim))[0]
    
    
    # combine ally and victim information
    scrubbers = allies + [victim]
    scrubber_volume = ally_capabilites + [victim_traffic]
    scrubber_paths = [edge_list[0] for edge_list in used_edges_list] + [[(u, v) for u, v in zip(victim_path[:-1], victim_path[1:])]]
    
    # keeping track
    used_nodes = [adversary]
    used_edges_simple = []
    
    # go through each path, and add to each node on the path how much traffic it will relay
    for scrubber_indx, scrubber_path in enumerate(scrubber_paths):
        for u, v in scrubber_path:
            G.nodes[v]["attack_vol"] += scrubber_volume[scrubber_indx]
            used_nodes.append(v)
            used_edges_simple.append((u, v))
            
    # now, go through the used edges and calculate the split percentage
    # TODO more elegant filling of list
    for node in used_nodes:
        # note the total attack volume it is relaying
        remaining = G.nodes[node]["attack_vol"]
        # note all its outward pointing edges that are used for carrying attack traffic
        # and the corresponding nodes with their total incoming attack traffic
        outward_flows = [(G.nodes[v]["attack_vol"], (u, v)) for u, v in G.out_edges(node) if (u, v) in used_edges_simple]
        # sort this from small to large by attack volumes
        outward_flows_sorted = sorted(outward_flows)
        # go through each next hop and calculate the percentage of traffic flowing their
        for next_hop_volume, (u, v) in outward_flows_sorted:
            G[u][v]["split_perc"] = min(next_hop_volume/G.nodes[node]["attack_vol"], remaining/G.nodes[node]["attack_vol"])
            remaining -= next_hop_volume
    return G
        
def color_graph(G_init, adversary, victim, allies, color = "orange", non_splitting_node_color = "black"):
    
    G = G_init.copy()
    
    # define a list for all special nodes
    special_nodes = [adversary, victim, *allies]
    
    # set the default attack volume for all nodes and edges
    for node in G.nodes:
        if G.nodes[node]["attack_vol"] != 0 and not node in special_nodes:
            # decide if the node on the path is actually splitting:
            if len(list(G.out_edges(node))) > 1:
                G.nodes[node]["color"] = color
            else:
                G.nodes[node]["color"] = non_splitting_node_color
    for u, v in G.edges:
        if G[u][v]["split_perc"] != 0:
            G[u][v]["color"] = color

    return G
            
G_final = apply_changes(G, final_changes, used_edges_list, used_splits)
G_final_w_splits = set_splits(G_final, adversary, used_edges_list, ATTACK_VOLUME, ALLIES_SCRUBBING_CAPABILITIES)
G_final_w_splits_w_color = color_graph(G_final_w_splits, adversary, victim, allies)

In [9]:
net_final = gen_pyvis_network(G_final_w_splits_w_color)
net_final.show("test.html")

### 5 - Test if Everything Works

In [202]:
# TODOs not all attack paths are covered...........................
# --> Problem in calculate_merge_split_cost

def test_attack_covered(G, adversary, victim):
    
    # get all attack paths
    all_attack_paths = list(nx.all_simple_paths(G, adversary, victim))
    
    # step through each node and check if every path has at least one splitter
    covered_by_splits = [False for _ in range(len(all_attack_paths))]
    for attack_path_indx, attack_path in enumerate(all_attack_paths):
        for node in attack_path:
            if "sub" in G.nodes[node]:
                covered_by_splits[attack_path_indx] = True
                break
    percentage_covered = 100 * sum(covered_by_splits)/len(covered_by_splits)
    
    print(f"{percentage_covered:.2f}% of attack paths covered!")
    
    return percentage_covered == 100

def ip_reachability(G, victim, allies):
    
    # get all the nodes that are not advertising
    nodes = [node for node in G.nodes if not node in [victim] + allies]
    
    # for each node, check that either the victim or an ally is reachable
    reachable = [False for _ in nodes]
    for node_indx, node in enumerate(nodes):
        if any(item in list(nx.descendants(G, node)) for item in [victim] + allies):
            reachable[node_indx] = True

    percentage_reachable = 100 * sum(reachable)/len(reachable)
    
    print(f"{percentage_reachable:.2f}% of nodes can reach the IP address in question!")
    
    return percentage_reachable == 100
        
test_attack_covered(G_final, adversary, victim)
ip_reachability(G_final, victim, allies)

0.00% of attack paths covered!
100.00% of nodes can reach the IP address in question!


True

### 6 - Questions

1. Regarding the Change from undirected to directed: Does that make sense for a BGP assignment? It is kinda crucial. For example one question I have is that many of the nodes have multiple ways to send traffic. And although this makes sense (depending on where traffic comes in from, it might exit somewhere specific), the traffic always enters at the same position, thus will always take only one exit? Maybe it makes sense to add somewhat of a "pruning" after making the graph directed, in which only one of the outwards pointing arrows is used; maybe depending on how many hops it takes to get to the destination (i.e, simulate AS_PATH feature in BGP decision process). --> I AM PRUNING RIGHT NOW; LETS TALK TO KATARINA

2. HUGE PROBLEM: Changing the edges in step 4 sometimes leads to non-connectivity to `victim` for many nodes

3. Some attack paths are not found?????? ANd merged???

TODO:
- decide the splits (percentage) and also test it